# **Construcción automática de texto (Pytorch-Ligthning)**
Andrey Duvan Rincon Torres

---

In [1]:
pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584 kB 4.9 MB/s 
     |████████████████████████████████| 140 kB 69.8 MB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
     |████████████████████████████████| 409 kB 53.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 54.0 MB/s 
     |████████████████████████████████| 271 kB 55.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [3]:
pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Librerias Nesesarias
import torch
import pandas as pd
import plotly.express as plx
import numpy as np
import plotly.graph_objects as go
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchmetrics.functional import accuracy
from sklearn.model_selection import train_test_split
from torchmetrics.functional import accuracy
CELoss = nn.CrossEntropyLoss()
# Para el texto
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Pre tratamiento de los datos**

In [6]:
# Descargar los datos
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")

--2022-05-27 23:45:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 209.85.145.128, 209.85.147.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2022-05-27 23:45:18 (126 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [7]:
# Crear estructura de datos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# crea predictores y etiqueta
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

## **Modelo**

In [8]:
class Model(nn.Module):
  # creamos la estructura de la red
  def __init__(self):
      super(Model,self).__init__()
      # Embeding de las palabras
      self.embedding = nn.Embedding(total_words, 100)
      # Red Bidireccional
      self.lstm_1 = nn.LSTM(100, 150, 1, batch_first=True, dropout = 0.2 , bidirectional = True)
      # red LSTM
      self.lstm_2 = nn.LSTM(300*(max_sequence_len - 1),100,1, batch_first=True)
      # red perceptron
      self.linear_1 = nn.Linear(100,1605)
      self.linear_2 = nn.Linear(1605,total_words)
      self.relu = nn.ReLU()
      self.sofmax = nn.Softmax()
  # definimos el comportamiento de las capas
  def forward(self, x):
      batch_size, channelsn = x.size()
      x = self.embedding(x)
      # layer LSTM bidirectional
      out, (h_n, c_n) = self.lstm_1(x)
      # layer LSTM
      out, (h_n, c_n) = self.lstm_2(out.reshape((batch_size,-1)))
      # capa de salida
      out = self.relu(self.linear_1(out))
      out = self.sofmax(self.linear_2(out))
      return out

## **Datos**

In [9]:
# Clase de los datos
class DataModule(pl.LightningDataModule):
  # Definimos un tamaño de lote en la calse
  def __init__(self, batch_size = 32):
      super(DataModule,self).__init__()
      self.batch_size = batch_size
  # Definimos el tratamiento de los datos
  def setup(self, stage=None):
    x, y = predictors, label
    # Conjunto de validacion
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.1, random_state = 0,shuffle=True)
    # Transformar en tensores
    self.train_dataset = TensorDataset(torch.tensor(x_train.astype(np.int32)),torch.tensor(y_train.astype(np.int32)))
    self.val_dataset = TensorDataset(torch.tensor(x_val.astype(np.int32)),torch.tensor(y_val.astype(np.int32)))
  # Iterable de entrenamiento
  def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch_size)
  # Iterable de validacion
  def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch_size)

## **Entrenamiento**

In [10]:
class Train(pl.LightningModule):
    # creamos la estructura de la red
    def __init__(self,model):
        super().__init__()
        self.model = model
   # Paso de entrenamiento
    def training_step(self, batch, batch_idx):
        loss,acc = self._shared_eval_step(batch, batch_idx)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss
    # Paso de validacion
    def validation_step(self, batch, batch_idx):
        loss,acc = self._shared_eval_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss
    # Funcion para evaluar el modelo y la perdida
    def _shared_eval_step(self,batch,batch_idx):
        x, y  = batch
        y_hat = self.model(x)
        loss = CELoss(y_hat, y.type(torch.LongTensor))
        acc = accuracy(y_hat, y.type(torch.LongTensor))
        return loss, acc
    # Configuracion del optimizador
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters())

##  Ajustar el modelo

In [ ]:
data_module = DataModule() # Ejecutamos modulo de datos
torch.manual_seed(0)
model = Model() # Ejecutamos modelo
trainer = pl.Trainer(max_epochs=200, progress_bar_refresh_rate=20) # Lamamos el entrenador
task = Train(model)
trainer.fit(task,data_module)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# Tablero de resultados
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

# **Generacion de texto**

In [ ]:
seed_text = "Who are you, so too cruel?"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.forward(torch.tensor(token_list.astype(np.float32))).detach().numpy().transpose()[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)